In [1]:
# Imports
import numpy as np
import pandas as pd
import re
import csv
import random
import math

In [2]:
#turns a sequence into a question format
# seq - whole dataset interger sequence
# l   - length of the question sequence
# c   - number of answer choices available
# p   - probability of missing element being last in the sequence
def toMCQuestion(seq,l,c,p):
    i = {}
    l = min(len(seq), l)
    
    #question
    a = random.randint(0,len(seq)-l)
    b = a+l-1
    s = seq[a:b]
    sd = math.ceil(np.std(s))
    if random.random() < p:
        m = len(s)-1              #use last element as missing
    else:
        m = random.randint(0,len(s)-1)   #random missing element
    
    #add sequence
    ans = s[m]
    s[m] = "?"    #set as missing
    i["sequence"] = s
    #i["sequence"] = ",".join([str(j) for j in s])
    
    #make randomly generated choices
    cs = []
    cs.append(ans)   #add right answer
    
    while len(cs) < c:       #add fake answers
        e = math.floor(ans+sd*random.uniform(-2,3))    #vary the answer choices using std and uniform randomness
        if e not in cs:
            cs.append(e)
    
    random.shuffle(cs)       #shuffle order
    i["options"] = cs
    
    #set answer to the missing element and add
    i["answer"] = np.where(np.array(cs)==ans)[0][0]
    
    return i


#test making questions from fibonacci
for i in range(10):
    print(toQuestion([1,1,2,3,5,8,13,21,34],5,4,0.75))

NameError: name 'toQuestion' is not defined

In [6]:
'''
  Assume the dataset is in vectorized 2d numeric array format where the whole set is given per row/sample
  Dataset is in the form .csv
  
  ex. 
      [ 9, 67, 14, 14, 65,  0, 23, 57, 49, 11, 22, 77, 43, 21, 60, 74, 56  .....] (length = 43)
      
'''

# read in dataset from a csv as a 2d array
def importData(filename):
    return list(csv.reader(open(filename)))


# create multiple choice questions from all of the samples in the dataset
# INPUTS:
# minQuest = minimum number of questions to use (-1 = all samples, > len(dataset) = +random selection )
# seqRange = length range of the starting sequence
# ansRange = number range of the possible answers
# lastProb = probability of the missing element being at the last element or somewhere else

def multiChoice(dataset, minQuest = -1, seqRange=[3,17],ansRange=[3,6],lastProb=0.85):
    qdat = []
    seqstr = []                #keep track of stringified saved sequences
    shufDat = dataset.copy()
    random.shuffle(shufDat)
    
    #go over each item
    i = 0
    for d in shufDat:
        #generate a question from the sample
        l = random.randint(seqRange[0],seqRange[1])
        c = random.randint(ansRange[0],ansRange[1])
        q = toMCQuestion(d,l,c,lastProb)
        
        q["id"] = i
        i+= 1
        
        seqstr.append(str(q["sequence"]))
        
        qdat.append(q)
        
        #enough questions, finish
        if minQuest > 0 and len(qdat) > minQuest:
            break
          
    #get random extra questions to fill quota
    leftover = minQuest - len(shufDat)
    if minQuest != -1 and leftover > 0:
        for i in range(leftover):
            
            d = random.choice(shufDat)  #get random sequence from the data
            
            #generate a question from the sample
            l = random.randint(seqRange)
            c = random.randint(ansRange)
            q = toMCQuestion(d,l,c,lastProb)

            q["id"] = i
            i+= 1

            #check if already in the dataset 
            if str(q["sequence"]) not in seqstr:
                qdat.append(q)
            
            #enough questions, finish
            if len(questions) > minQuest:
                break
    
    #return multiple choice set
    return qdat

In [7]:
#test on arithmetic sequences
# makes a sequence that starts at a random value and adds it by another value consistently 
# equation: [A_n = A_(n-1)+b; A_0, b = ?? ]
def testArith(n,k):
    d = []
    for i in range(n):
        kr = random.randint(5,k)
        a = random.randint(0,5)
        b = random.randint(1,20)
        s = []
        for j in range(kr):
            s.append(a+j*b)      
        d.append(s)
    return d

td = testArith(30,13)
print(td)

[[5, 18, 31, 44, 57, 70, 83, 96], [0, 20, 40, 60, 80, 100, 120], [2, 18, 34, 50, 66, 82, 98, 114, 130, 146], [1, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45], [4, 8, 12, 16, 20, 24, 28, 32], [0, 9, 18, 27, 36, 45, 54, 63, 72], [2, 18, 34, 50, 66, 82], [0, 4, 8, 12, 16, 20, 24, 28], [1, 21, 41, 61, 81, 101, 121], [5, 13, 21, 29, 37, 45, 53, 61, 69, 77, 85, 93], [4, 15, 26, 37, 48, 59, 70, 81, 92, 103, 114], [1, 14, 27, 40, 53, 66, 79, 92, 105, 118, 131, 144, 157], [5, 23, 41, 59, 77, 95, 113, 131, 149, 167, 185], [0, 2, 4, 6, 8], [4, 8, 12, 16, 20, 24, 28, 32, 36, 40], [2, 17, 32, 47, 62, 77, 92], [5, 23, 41, 59, 77, 95, 113, 131, 149, 167, 185, 203, 221], [0, 18, 36, 54, 72, 90, 108], [0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200], [5, 22, 39, 56, 73, 90, 107, 124, 141, 158], [3, 21, 39, 57, 75], [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50], [1, 21, 41, 61, 81, 101, 121, 141, 161], [0, 12, 24, 36, 48, 60, 72, 84, 96], [5, 8, 11, 14, 17, 20], [4, 19, 34, 49, 64, 79], [4, 20, 36, 52, 68], 

In [9]:
mcq = multiChoice(td)
print(mcq)

[{'sequence': [0, 20, 40, 60, 80, '?'], 'options': [133, 163, 34, 98, 100, 101], 'answer': 4, 'id': 0}, {'sequence': [1, 14, 27, 40, 53, 66, 79, 92, 105, 118, 131, '?'], 'options': [240, 144, 259], 'answer': 1, 'id': 1}, {'sequence': [8, '?'], 'options': [15, 9, 10, 17, 8, 12], 'answer': 5, 'id': 2}, {'sequence': [61, '?'], 'options': [83, 81, 103], 'answer': 1, 'id': 3}, {'sequence': [5, 18, 31, 44, 57, 70, '?'], 'options': [83, 123, 102, 57, 112], 'answer': 0, 'id': 4}, {'sequence': [5, 23, 41, 59, 77, 95, 113, 131, '?', 167, 185, 203], 'options': [149, 82, 250, 164, 172, 323], 'answer': 0, 'id': 5}, {'sequence': [2, 18, 34, 50, 66, 82, 98, 114, '?'], 'options': [130, 244, 166], 'answer': 0, 'id': 6}, {'sequence': [4, 18, 32, 46, 60, 74, '?', 102, 116], 'options': [126, 88, 63], 'answer': 1, 'id': 7}, {'sequence': [5, 8, 11, 14, '?'], 'options': [17, 15, 21, 29, 13, 22], 'answer': 0, 'id': 8}, {'sequence': [0, 18, 36, 54, 72, '?'], 'options': [90, 174, 159, 55], 'answer': 0, 'id': 9}

In [11]:
type(mcq[0]['options'])

list